# Проект. Проблема дорожно-транспортных происшествий в России. 
# Исследование данных и изучение взаимосвязей. С чем связана аварийность на дорогах.
## На примере Кировской области

## 1. Выполним предобработку для удобства работы с данными: выбрать оптимальные названия столбцов и типы данных. Понадобится дать ответы на вопросы заказчика.

### 1.1  Вопросы заказчика
- Как распределено количество участников ДТП и почему? Встречаются ли аномальные значения или выбросы? Если да, то с чем они могут быть связаны? Для числа участников найдите наиболее типичное значение. По желанию можете проверить распределения и других столбцов.

- Предположите, между какими столбцами в датасете Kirovskaya_oblast.csv высокая корреляция. Проверьте своё предположение.

- Как связаны категории аварий и погодные условия?

- Постройте процентную разбивку аварий по видам освещённости. При этом учитывайте пол участника и сделайте расчёты для мужчин и женщин отдельно.

- Исследуйте, чем отличаются аварии без пострадавших от тех, в которых был один пострадавший или более.

- Сделайте общий вывод о связи аварийности с другими факторами. Какие рекомендации вы можете дать заказчику?

## 2.  Описание данных
Датасет `Kirovskaya_oblast.csv` содержит информацию о ДТП в Кировской области:

- geometry.coordinates — координаты ДТП;
- id — идентификатор ДТП;
- properties.tags — тег происшествия;
- properties.light — освещённость;
- properties.point.lat — широта;
- properties.point.long — долгота;
- properties.nearby — ближайшие объекты;
- properties.region — регион;
- properties.scheme — схема ДТП;
- properties.address — ближайший адрес;
- properties.weather — погода;
- properties.category — категория ДТП;
- properties.datetime — дата и время ДТП;
- properties.injured_count — число пострадавших;
- properties.parent_region — область;
- properties.road_conditions — состояние покрытия;
- properties.participants_count — число участников;
- properties.participant_categories — категории участников.

Датасет `Kirovskaya_oblast_participiants.csv` хранит сведения об участниках:
- role — роль;
- gender — пол;
- violations — какие правила дорожного движения были нарушены конкретным участником;
- health_status — состояние здоровья после ДТП;
- years_of_driving_experience — число лет опыта;
- id — идентификатор ДТП.

## 3. Загрузим данные и выполним небольшую предобработку

## 4.  Исследование данных

- 3.1 Как распределено количество участников ДТП и почему? Встречаются ли аномальные значения или выбросы? Если да, то с чем они могут быть связаны? Для числа участников найдите наиболее типичное значение. По желанию можете проверить распределения и других столбцов.
​
- 3.2 Предположите, между какими столбцами в данных высокая корреляция? Проверьте своё предположение.
​
- 3.3 Как связаны категории аварий и погодные условия?
​
- 3.4 Постройте процентную разбивку аварий по видам освещённости. При этом учитывайте пол участника и сделайте расчёты для мужчин и женщин отдельно.
​
- 3.5 Исследуйте, чем отличаются аварии без пострадавших от тех, в которых был один пострадавший или более.
​
- 3.6 Сделайте общий вывод о связи аварийности с другими факторами. Какие рекомендации вы можете дать заказчику?
​


In [ ]:
# Импортируем библиотеки
import pandas as pd

# Загружаем библиотеки для визуализации данных
import matplotlib.pyplot as plt
import seaborn as sns

# Загружаем библиотеку для расчёта коэффициента корреляции phi_k
from phik import phik_matrix

df_info = pd.read_csv('https://code.s3.yandex.net/datasets/Kirovskaya_oblast.csv')
df_partic = pd.read_csv('https://code.s3.yandex.net/datasets/Kirovskaya_oblast_participiants.csv')

## Для удобства изменим названия столбцов первого датафрейма, используя код ниже:
df_info.columns = [c.replace('properties.', '') for c in df_info.columns]

## Знакомимся с данными датасета


In [2]:
# Выведем первые строки методом head(), а информацию о датафрейме методом info():
df_info.info()
df_info.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14517 entries, 0 to 14516
Data columns (total 18 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   geometry.coordinates    14517 non-null  object 
 1   id                      14517 non-null  int64  
 2   tags                    14517 non-null  object 
 3   light                   14517 non-null  object 
 4   point.lat               14485 non-null  float64
 5   point.long              14485 non-null  float64
 6   nearby                  14517 non-null  object 
 7   region                  14517 non-null  object 
 8   scheme                  13380 non-null  float64
 9   address                 13843 non-null  object 
 10  weather                 14517 non-null  object 
 11  category                14517 non-null  object 
 12  datetime                14517 non-null  object 
 13  injured_count           14517 non-null  int64  
 14  parent_region           14517 non-null

,geometry.coordinates,id,tags,light,point.lat,point.long,nearby,region,scheme,address,weather,category,datetime,injured_count,parent_region,road_conditions,participants_count,participant_categories
0,"[47.875603, 57.24379]",1983180,Дорожно-транспортные происшествия,Светлое время суток,57.243790,47.875603,[],Яранский район,600.0,Р-176 Вятка Чебоксары - Йошкар-Ола - Киров - С...,['Дождь'],Опрокидывание,2017-07-01 18:00:00,1,Кировская область,['Мокрое'],3,['Все участники']
1,"[47.87903, 57.304807]",2889433,Дорожно-транспортные происшествия,Светлое время суток,57.304807,47.879030,"['Административные здания', 'Нерегулируемый пе...",Яранский район,710.0,"г Яранск, ул Кирова, 10",['Ясно'],Наезд на пешехода,2023-09-12 17:10:00,1,Кировская область,"['Сухое', 'Отсутствие, плохая различимость гор...",2,"['Все участники', 'Пешеходы']"
2,"[47.840781, 57.297156]",2591208,Дорожно-транспортные происшествия,Сумерки,57.297156,47.840781,"['Жилые дома индивидуальной застройки', 'Нерег...",Яранский район,NaN,"г Яранск, ул Чапаева, 80",['Пасмурно'],Съезд с дороги,2021-07-02 21:25:00,1,Кировская область,['Мокрое'],1,['Все участники']
3,"[47.834365, 57.244775]",2577639,Дорожно-транспортные происшествия,Светлое время суток,57.244775,47.834365,['Жилые дома индивидуальной застройки'],Яранский район,200.0,"м Знаменка, ул Кирова, 15",['Пасмурно'],Столкновение,2021-05-31 18:55:00,1,Кировская область,['Сухое'],2,"['Все участники', 'Мотоциклисты']"
4,"[47.968197, 57.357738]",1981026,Дорожно-транспортные происшествия,Светлое время суток,57.357738,47.968197,['Нерегулируемый перекрёсток неравнозначных ул...,Яранский район,NaN,"с/п Никольское, Киров-Советск- Яранск - подъез...",['Ясно'],Опрокидывание,2018-05-16 16:25:00,2,Кировская область,"['Сухое', 'Отсутствие, плохая различимость гор...",2,['Все участники']


# РАЗДЕЛ РЕДАКТИРОВАТЬ!!!!!

Датасет rest_info содержит ____ столбцов и ____ строк, в которых содержится информация о ________.

После первичного анализа данных можно сделать следующие выводы:

Названия столбцов корректны, исполнены в едином стиле, в нижнем регистре.
Представленные в столбцах __________ данные содержат числовые значения и хранятся в типах данных int64 или float64. Столбец chain содержит значения 1 или 0 — размерность этих данных можно оптимизировать.
Значения в столбцах __________ представлены данными типа object. Корректировка типов данных не требуется.
Пропуски содержатся только в столбах __________. Также проверим и другие столбцы: в них могут встречаться значения-индикаторы, которыми заменены пропуски.
Судя по первому знакомству с данными, значения в столбцах соответствуют своему описанию.

In [3]:
df_partic.info()
df_partic.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31235 entries, 0 to 31234
Data columns (total 6 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   role                         31235 non-null  object 
 1   gender                       30387 non-null  object 
 2   violations                   31235 non-null  object 
 3   health_status                31135 non-null  object 
 4   years_of_driving_experience  16909 non-null  float64
 5   id                           31235 non-null  int64  
dtypes: float64(1), int64(1), object(4)
memory usage: 1.4+ MB


,role,gender,violations,health_status,years_of_driving_experience,id
0,Водитель,Мужской,['Несоответствие скорости конкретным условиям ...,"Раненый, находящийся (находившийся) на амбулат...",26.0,1983180
1,Водитель,Мужской,[],Не пострадал,34.0,2889433
2,Пассажир,Мужской,[],"Раненый, находящийся (находившийся) на амбула...",NaN,2591208
3,Пассажир,Мужской,[],"Раненый, находящийся (находившийся) на амбула...",NaN,2591208
4,Водитель,Мужской,[],Не пострадал,27.0,2577639
